In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [2]:
# Load the dataset
dat = pd.read_csv("Final_Data.csv",low_memory=False)

In [3]:
# Define categorical and numerical columns
cat_cols = ['STRUCTURE_KIND_043A', 'STRUCTURE_TYPE_043B', 'DECK_STRUCTURE_TYPE_107', 'LOWEST_RATING']
num_cols = ['ADT_029', 'MAX_SPAN_LEN_MT_048', 'IMP_LEN_MT_076', 'DECK_AREA', 'TIC', 'Installation_Year']

# One-hot encode categorical columns
cat_encoded = pd.get_dummies(dat, columns=cat_cols, dtype=float)

# Initialize the MinMaxScaler for numerical columns
scaler = MinMaxScaler()

# Apply MinMax scaling to the numerical features
cat_encoded[num_cols] = scaler.fit_transform(cat_encoded[num_cols])

test_data = cat_encoded[(cat_encoded['CollectionYear'].isin([2021, 2022]))]

# Filter the training dataset
train_data = cat_encoded[(cat_encoded['CollectionYear'] > 2010) &
                        (cat_encoded['CollectionYear'] <= 2020)]

# Display summary
print("Training Data:")
print(train_data.shape)
print("\nTesting Data (Non-Identical Ratings):")
print(test_data.shape)


Training Data:
(99975, 55)

Testing Data (Non-Identical Ratings):
(20941, 55)


In [4]:
n_structures_per_query = 10
n_queries = 20

In [5]:
# Assuming 'train_data' is predefined somewhere in your environment
total_bridges_in_train_data = train_data['STRUCTURE_NUMBER_008'].nunique()

# Calculate the number of unique bridges in each cluster for proportional weights calculation
cluster_counts = train_data.groupby('hierarchical_cluster')['STRUCTURE_NUMBER_008'].nunique()
proportional_weights = cluster_counts / total_bridges_in_train_data
print("Proportional weights for each cluster:", proportional_weights)

# Set the total number of samples you want to draw
total_samples = n_structures_per_query

# Calculate the number of bridges to sample from each cluster based on proportional weights
bridges_per_cluster = (proportional_weights * total_samples).round().astype(int)

# Ensure the sum of samples equals exactly 20 by adjusting if necessary
while bridges_per_cluster.sum() != total_samples:
    print("Adjusting since total is not 20:")
    if bridges_per_cluster.sum() > total_samples:
        max_index = bridges_per_cluster.idxmax()
        bridges_per_cluster[max_index] -= 1
        print(f"Reduced by 1 in cluster {max_index}, new distribution:", bridges_per_cluster)
    elif bridges_per_cluster.sum() < total_samples:
        min_index = bridges_per_cluster.idxmin()
        bridges_per_cluster[min_index] += 1

# Ensure at least one sample from each cluster if possible
bridges_per_cluster = bridges_per_cluster.clip(lower=1)

# Sample randomly from each cluster proportionate to the weights
selected_bridges = []
for cluster, count in bridges_per_cluster.items():
    if count > 0:  # Check if there are bridges to sample
        cluster_data = train_data[train_data['hierarchical_cluster'] == cluster]
        sample_ids = cluster_data['STRUCTURE_NUMBER_008'].drop_duplicates().sample(n=count, random_state=42)
        selected_bridges.extend(sample_ids.tolist())

# Create initial_set by filtering all records of the selected bridge IDs
initial_set = train_data[train_data['STRUCTURE_NUMBER_008'].isin(selected_bridges)]

# Create pool_set by filtering out the records of the selected bridge IDs
pool_set = train_data[~train_data['STRUCTURE_NUMBER_008'].isin(selected_bridges)]


Proportional weights for each cluster: hierarchical_cluster
1    0.004335
2    0.163862
3    0.109213
4    0.684855
5    0.002125
6    0.000425
7    0.035186
Name: STRUCTURE_NUMBER_008, dtype: float64


## data preprocessing

In [6]:
print(type(proportional_weights))


<class 'pandas.core.series.Series'>


In [7]:
pool_set.shape

(99863, 55)

In [8]:
initial_set = initial_set.drop(columns=['CollectionYear'])
pool_set = pool_set.drop(columns=['CollectionYear'])
test_data = test_data.drop(columns=['CollectionYear'])

In [9]:
initial_set.loc[:, 'NEXT_LOWEST_RATING'] = initial_set['NEXT_LOWEST_RATING'] - 3
pool_set.loc[:, 'NEXT_LOWEST_RATING'] = pool_set['NEXT_LOWEST_RATING'] - 3
test_data.loc[:, 'NEXT_LOWEST_RATING'] = test_data['NEXT_LOWEST_RATING'] - 3

In [10]:
print(initial_set.shape)
print(pool_set.shape)
print(test_data.shape)

(112, 54)
(99863, 54)
(20941, 54)


## Fast KELMOR

In [11]:
def incomplete_cholesky(g_row,g_diag,K, S, N, epsilon = 1e-5):

  pi = list(range(N))
  P = np.zeros([S,N])
  D = np.copy(g_diag())
  err = np.sum(np.abs(D))

  s = 0

  while(s < S) and (err > epsilon):
    i = s + np.argmax([D[pi[j]] for j in range(s,N)])

    # line 6 : swap pi[s] and pi[i]

    tmp = pi[s]
    pi[s] = pi[i]
    pi[i] = tmp

    # line 7 :
    P[s,pi[s]] = np.sqrt(D[pi[s]])
    KX = g_row(pi[s])
    for i in range(s+1, N):
      if s > 0:
        inner_p = np.inner(P[:s,pi[s]], P[:s,pi[i]])
      else:
        inner_p = 0

      P[s,pi[i]] = (KX[pi[i]] - inner_p) / P[s,pi[s]]
      D[pi[i]] -=  pow(P[s,pi[i]],2)
    err = np.sum([D[pi[i]] for i in range(s+1,N)])
    s = s + 1

  P = P[:s,:]

  return P

In [12]:
import numpy as np
from scipy.stats import kendalltau, somersd
from sklearn.metrics.pairwise import pairwise_kernels


class kelmor():
    def __init__(self, kernel, C):
        self.kernel = kernel
        self.C = C

    def fit(self, X, y):
        self.X = X
        self.y = y
        N, F = X.shape
        self.t = np.array([[(j-q)**2 for j in range(7)] for q in range(7)])
        T = self.t[y, :]
        K = pairwise_kernels(X, metric=self.kernel)
        g_row = lambda i: K[i, :]
        g_diag = lambda: np.diag(K).copy()
        N = K.shape[0]
        S = 500
        P = incomplete_cholesky(g_row, g_diag, K, S, N, epsilon=1e-5)
        P = np.transpose(P)
        z = self.C * T
        u = np.matmul(np.transpose(P), T)
        s = np.matmul(np.linalg.inv(np.eye(P.shape[1]) + self.C * np.matmul(np.transpose(P), P)), u)
        self.beta = z - self.C**2 * np.matmul(P, s)
        return self

    def inference(self, X):
        K = pairwise_kernels(X, self.X, metric=self.kernel)
        fx = np.dot(K, self.beta)
        self.y_hat = np.argmin(np.linalg.norm(fx[:, None] - self.t, ord=1, axis=2), axis=1)
        NR = -np.linalg.norm(fx[:, None] - self.t, ord=1, axis=2)
        self.probs = self.soft_max(NR)
        return self.y_hat, self.probs

    def soft_max(self, NR):
        P = np.exp(NR) / (np.sum(np.exp(NR), axis=1)[:, np.newaxis])
        return P

In [13]:
kel = kelmor(kernel = "linear",C = 5)

In [14]:
# Define the rps_value function (correct version dont touch)
def rps_value(y_pred_prob, y_true):
    """
    Calculate Ranked Probability Score (RPS) for ordinal predictions.

    Parameters
    ----------
    y_pred_prob : numpy array
        Array of shape (num_samples, num_categories) containing predicted probabilities for each ordinal category.
    y_true : numpy array
        Array of shape (num_samples,) containing true ordinal category values.

    Returns
    -------
    float
        Mean RPS value across all samples.
    """
    # Convert y_true to numpy array for positional indexing
    y_true = np.array(y_true)
    num_samples, num_cat = y_pred_prob.shape
    # Ensure the true labels match the number of samples
    if num_samples != len(y_true):
        raise ValueError(f"Number of samples does not match: {num_samples}, {len(y_true)}")


    # Ensure the true labels match the number of samples
    if num_samples != len(y_true):
        raise ValueError(f"Number of samples does not match: {num_samples}, {len(y_true)}")

    # Compute CDFs for predicted probabilities
    y_pred_prob_cdf = np.cumsum(y_pred_prob, axis=1)

    # Compute CDFs for true labels
    y_true_cdf = np.zeros_like(y_pred_prob_cdf)
    for k in range(num_samples):
        y_true_cdf[k, :] = [(j >= y_true[k]) * 1 for j in range(num_cat)]

    # Compute RPS for each sample
    rps_per_sample = np.sum((y_pred_prob_cdf - y_true_cdf) ** 2, axis=1) / (num_cat - 1)

    # Return the mean RPS value across all samples
    return np.round(np.mean(rps_per_sample),4)


## training

In [15]:
# Define your feature columns (excluding target, cluster, and structure number)
feature_columns = [col for col in initial_set.columns if col not in ['NEXT_LOWEST_RATING', 'hierarchical_cluster', 'STRUCTURE_NUMBER_008']]

# Prepare training and testing data
X_train = initial_set[feature_columns]
y_train = initial_set['NEXT_LOWEST_RATING'].values

X_test = test_data[feature_columns]
y_test = test_data['NEXT_LOWEST_RATING'].values

print('X_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print('X_test shape:', X_test.shape)
print('y_test shape:', y_test.shape)

# Train the initial model
kel.fit(X_train.values, y_train)

# Generate predicted probabilities for the initial training set
_, y_pred_prob_train = kel.inference(X_train.values)


X_train shape: (112, 51)
y_train shape: (112,)
X_test shape: (20941, 51)
y_test shape: (20941,)


In [16]:
# Calculate RPS for the initial training set
initial_rps = rps_value(y_pred_prob_train, y_train)
print(f"Initial Ranked Probability Score (RPS): {initial_rps:.4f}")

Initial Ranked Probability Score (RPS): 0.0431


In [17]:
def entropy_sampling_for_structure(kel, pool_set, feature_columns, n_samples, cluster_proportional):
    cluster_entropy = {}
    for cluster in pool_set['hierarchical_cluster'].unique():
        subset = pool_set[pool_set['hierarchical_cluster'] == cluster]
        entropies = []
        for structure_number in subset['STRUCTURE_NUMBER_008'].unique():
            structure_subset = subset[subset['STRUCTURE_NUMBER_008'] == structure_number]
            _, probabilities = kel.inference(structure_subset[feature_columns].values)
            entropy = -np.sum(probabilities * np.log(probabilities + 1e-5), axis=1).mean()
            entropies.append((structure_number, entropy))

        # Sort structures in this cluster by descending entropy
        entropies.sort(key=lambda x: x[1], reverse=True)
        cluster_entropy[cluster] = entropies

    # Select structures from each cluster based on their entropy and proportional needs
    selected_structures = []
    for cluster, entropies in cluster_entropy.items():
        n_to_select = max(1, int((n_samples * cluster_proportional.get(cluster, 0))+.5))
        selected_structures.extend([structure[0] for structure in entropies[:n_to_select]])

    return selected_structures[:n_samples]


In [18]:
cluster_counts = pool_set['hierarchical_cluster'].value_counts()
total_bridges = cluster_counts.sum()
cluster_proportional = (cluster_counts / total_bridges).to_dict()

In [19]:
import pandas as pd
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, classification_report

def run_sampling_method(
    kel,
    X_train, y_train,
    pool_set,
    X_test, y_test,
    n_queries,
    feature_columns,
    n_structures_per_query
):
    # Initialize an empty list to store metrics (RPS ,precision, recall, F1 score)
    metrics_data = []

    # Initial model training and evaluation
    kel.fit(X_train.values, y_train)
    _, y_pred_probs = kel.inference(X_test.values)
    y_pred = np.argmax(y_pred_probs, axis=1)  # Get the predicted class labels

    # Initial metrics calculation
    initial_rps = rps_value(y_pred_probs, y_test)
    initial_precision, initial_recall, initial_f1, _ = precision_recall_fscore_support(
        y_test, y_pred, average='weighted'  
    )

    # Store initial metrics
    metrics_data.append({
        'Query': 0,
        'RPS': initial_rps,
        'Precision': initial_precision,
        'Recall': initial_recall,
        'F1 Score': initial_f1
    })

    # Print initial metrics and confusion matrix
    print(f"Initial Metrics - RPS: {initial_rps:.4f}, "
          f"Precision: {initial_precision:.4f}, Recall: {initial_recall:.4f}, F1 Score: {initial_f1:.4f}")

    # Calculate and print initial confusion matrix
    initial_cm = confusion_matrix(y_test, y_pred)
    print(f"Initial Confusion Matrix:\n", initial_cm)

    for query_index in range(1, n_queries + 1):
        print(f"\n--- Query {query_index} ---")

        # Calculate cluster proportions for entropy sampling
        cluster_counts = pool_set['hierarchical_cluster'].value_counts()
        total_bridges = cluster_counts.sum()
        cluster_proportional = (cluster_counts / total_bridges).to_dict()

        # Get high entropy structure numbers from the entropy sampling method
        high_entropy_structure_numbers = entropy_sampling_for_structure(
            kel, pool_set, feature_columns, n_structures_per_query, cluster_proportional
        )

        # Check if new samples are selected based on entropy
        if not high_entropy_structure_numbers:
            print("No new samples selected based on entropy. Stopping active learning.")
            break

        # Select new samples based on high entropy structure numbers
        new_samples = pool_set[pool_set['STRUCTURE_NUMBER_008'].isin(high_entropy_structure_numbers)]
        count = new_samples.shape[0]
        print(f"Selected {len(high_entropy_structure_numbers)} structures with total {count} samples")

        # Update pool set by removing selected samples
        pool_set = pool_set[~pool_set['STRUCTURE_NUMBER_008'].isin(high_entropy_structure_numbers)].reset_index(drop=True)

        # Extract features and labels from new_samples
        X_new = new_samples[feature_columns]
        y_new = new_samples['NEXT_LOWEST_RATING'].values

        # Append new samples to the training set
        X_train = pd.concat([X_train, X_new], ignore_index=True)
        y_train = np.concatenate([y_train, y_new])

        # Retrain the model with the updated training set
        kel.fit(X_train.values, y_train)

        # Evaluate the model on the test set
        _, y_pred_probs = kel.inference(X_test.values)
        y_pred = np.argmax(y_pred_probs, axis=1)  # Get the predicted class labels

        # Calculate metrics for this query
        rps = rps_value(y_pred_probs, y_test)
        precision, recall, f1, _ = precision_recall_fscore_support(
            y_test, y_pred, average='weighted'  
        )

        # Store the metrics
        metrics_data.append({
            'Query': query_index,
            'RPS': rps,
            'Precision': precision,
            'Recall': recall,
            'F1 Score': f1
        })

        # Print the metrics for this query
        print(f"Query {query_index}: RPS = {rps:.4f}, "
              f"Precision = {precision:.4f}, Recall = {recall:.4f}, F1 Score = {f1:.4f}")

        # Calculate and print confusion matrix for the current query
        cm = confusion_matrix(y_test, y_pred)
        print(f"Confusion Matrix for Query {query_index}:\n", cm)

    # Convert metrics_data to a DataFrame
    metrics_df = pd.DataFrame(metrics_data)

    # Save the results to an Excel file
    metrics_df.to_excel('active_learning_metrics.xlsx', index=False)

    # Return metrics DataFrame, true labels, and predictions
    return metrics_df, y_test, y_pred


# Running the sampling method
metrics_df, y_test, y_pred = run_sampling_method(
    kel=kel, 
    X_train=X_train,
    y_train=y_train,
    pool_set=pool_set,
    X_test=X_test,
    y_test=y_test,
    n_queries=n_queries,
    feature_columns=feature_columns,
    n_structures_per_query=n_structures_per_query
)

# Generate and print the classification report
report = classification_report(y_test, y_pred)
print("\nClassification Report:\n", report)

# Save the classification report as a DataFrame
report_dict = classification_report(y_test, y_pred, output_dict=True)
report_df = pd.DataFrame(report_dict).transpose()
report_df.to_excel("classification_report.xlsx")


Initial Metrics - RPS: 0.0340, Precision: 0.7904, Recall: 0.7902, F1 Score: 0.7896
Initial Confusion Matrix:
 [[ 154  108   14    8    1    0    1]
 [ 117 1320  272   27    6    1    0]
 [   2  365 4088  944   32    2    2]
 [   0   17  644 4559  568   15    6]
 [   0    0    6  143 4179  409   23]
 [   0    0    1    4  415 1474  119]
 [   0    0    0    0    0  122  773]]

--- Query 1 ---
Selected 10 structures with total 68 samples
Query 1: RPS = 0.0286, Precision = 0.8199, Recall = 0.8191, F1 Score = 0.8181
Confusion Matrix for Query 1:
 [[ 223   36   22    4    0    0    1]
 [   0 1571  151   15    6    0    0]
 [   0  233 4829  322   46    3    2]
 [   0    0  789 4264  733   19    4]
 [   0    0    0  573 3822  341   24]
 [   0    0    0    0  205 1695  113]
 [   0    0    0    0    0  147  748]]

--- Query 2 ---
Selected 10 structures with total 62 samples
Query 2: RPS = 0.0149, Precision = 0.9142, Recall = 0.9131, F1 Score = 0.9133
Confusion Matrix for Query 2:
 [[ 224   35   